In [5]:
# Note that this code uses libraries from the TENNLab Neuromorphic Framework
import eons
import neuro
import risp
from torchvision import datasets, transforms
from random import randint
import json

# some values
NUM_INPUTS = 784
NUM_OUTPUTS = 10
NUM_HIDDEN = 200
NUM_SYNAPSES = 750
NUM_NEURONS = NUM_INPUTS+NUM_OUTPUTS+NUM_HIDDEN 
MOA = neuro.MOA()
MOA.seed(132312, '') # NEED THESE OTHERWISE DOESN'T RANDOMIZE

DATASET_CAP = 50 # for debugging

# eons and encoding stuff
POP_SIZE = 100
RATE_ENCODING_INTERVAL = 20

In [6]:
# Fashion MNIST dataset
transform = transforms.ToTensor() # scale down to 0-1

train_dataset = datasets.FashionMNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.FashionMNIST(root='./data', train=False, transform=transform, download=True)

In [7]:
# preprocessing
train_dataset = [(train_dataset[i][0].flatten().tolist(), train_dataset[i][1]) for i in range(len(train_dataset))][:DATASET_CAP]
test_dataset = [(test_dataset[i][0].flatten().tolist(), test_dataset[i][1]) for i in range(len(test_dataset))]

In [8]:
risp_config = { #https://bitbucket.org/neuromorphic-utk/framework/src/93a1bbba2c0a31ca8f036ff2a89a4d6ffae5cec3/processors/risp/README.md#markdown-header-params
  "min_weight": -1,
  "max_weight": 1,
  "min_threshold": -1,
  "max_threshold": 1,
  "min_potential": -1,
  "max_delay": 10,
  "discrete": False
}

proc = risp.Processor(risp_config)

# configure eons
eons_param = {
    "starting_nodes": 4,
    "starting_edges": 10,
    "merge_rate": 0,
    "population_size": POP_SIZE,
    "multi_edges": 0,
    "crossover_rate": 0.5,
    "mutation_rate": 0.9,
    "selection_type": "tournament",
    "tournament_size_factor": 0.1,
    "tournament_best_net_factor": 0.9,
    "random_factor": 0.05,
    "num_mutations": 5,
    "node_mutations": { "Threshold": 1.0 },
    "net_mutations": { },
    "edge_mutations": { "Weight": 0.65 , "Delay": 0.35 },
    "num_best" : 2
}
eons_inst = eons.EONS(eons_param)

In [9]:
# create the network from which eons will base the others
net = neuro.Network()
net.set_properties(proc.get_network_properties())

# create input, output and hidden neurons
for i in range(NUM_INPUTS):
    node = net.add_node(i)
    node.set("Threshold", 1)
    net.randomize_node_properties(MOA, node)
    net.add_input(i)

for i in range(NUM_INPUTS, NUM_INPUTS+NUM_OUTPUTS):
    node = net.add_node(i)
    node.set("Threshold", 1)
    net.randomize_node_properties(MOA, node)
    net.add_output(i)

for i in range(NUM_INPUTS+NUM_OUTPUTS, NUM_INPUTS+NUM_OUTPUTS+NUM_HIDDEN):
    node = net.add_node(i)
    net.randomize_node_properties(MOA, node)

# create synapses randomly
for i in range(NUM_SYNAPSES):
    synapse = net.add_or_get_edge(randint(0, NUM_NEURONS-1), randint(0, NUM_NEURONS-1)) # randomly get 2 ids
    net.randomize_edge_properties(MOA, synapse)

In [10]:
# IGNORE THIS # IGNORE THIS # IGNORE THIS # IGNORE THIS # IGNORE THIS # IGNORE THIS # IGNORE THIS # IGNORE THIS # IGNORE THIS 

#encoder_params = {
#    "encoder_specs" : {
#            "default" : {
#                "bins"          : n_inputs,
#                "max_spikes"    : 1, 
#                "ov_interval"   : 2,
#                "intrabin"      : "spike_count",
 #               "interbin"      : "simple",
#                "spike_min"     : 1,
#               "spike_max"     : 1
 #           }
 #       },
 #   "dmin" : [0],
 #   "dmax" : [1],
 #   "use_encoders" : ["default"]
#}


# Create the encoder with the appropriate encoder parameters.  
# The first two parameters for Encoder Array correspond to the minimum and maximum values that can be obtained. 
#encoder = neuro.EncoderArray(encoder_params)

## OTHER

# Run one sample through our network
#encoder = neuro.SpikeEncoder('rate')
#encoder.set_overall_interval(10, True)

#encoder.get_spikes(1, 0, 1)
with open('dump.json', 'w') as f:
    f.write(str(net))

In [11]:
# Set up encoder, we'll use rate encoding

settings_encoder = {
    'dmin': [0 for i in range(784)],
    'dmax': [1 for i in range(784)],
    'default_interval': RATE_ENCODING_INTERVAL,
    "encoders": [ "rate" ]
}

encoder = neuro.EncoderArray(settings_encoder)

# debug - looks like encoder array automatically creates our spikes at the given rate for each id
# I'm assuming it just guesses input ids based on element of our input list index?
spikes = encoder.get_spikes(test_dataset[0][0])
for s in spikes:
    print(f'({s.id} at {s.time:.2f})', end=', ')

(215 at 0.00), (216 at 0.00), (219 at 0.00), (221 at 0.00), (221 at 6.89), (221 at 13.78), (237 at 0.00), (238 at 0.00), (240 at 0.00), (240 at 9.44), (240 at 18.89), (241 at 0.00), (241 at 3.04), (241 at 6.07), (241 at 9.11), (241 at 12.14), (241 at 15.18), (241 at 18.21), (242 at 0.00), (249 at 0.00), (249 at 2.14), (249 at 4.29), (249 at 6.43), (249 at 8.57), (249 at 10.71), (249 at 12.86), (249 at 15.00), (249 at 17.14), (249 at 19.29), (265 at 0.00), (268 at 0.00), (268 at 2.90), (268 at 5.80), (268 at 8.69), (268 at 11.59), (268 at 14.49), (268 at 17.39), (269 at 0.00), (269 at 1.78), (269 at 3.57), (269 at 5.35), (269 at 7.13), (269 at 8.92), (269 at 10.70), (269 at 12.48), (269 at 14.27), (269 at 16.05), (269 at 17.83), (269 at 19.62), (270 at 0.00), (270 at 2.32), (270 at 4.64), (270 at 6.95), (270 at 9.27), (270 at 11.59), (270 at 13.91), (270 at 16.23), (270 at 18.55), (275 at 0.00), (275 at 11.59), (276 at 0.00), (276 at 2.74), (276 at 5.48), (276 at 8.23), (276 at 10.97), 

In [12]:
# Apply spikes as an example
proc.load_network(net)

for i in range(net.num_nodes()):
    proc.track_neuron_events(i)

proc.apply_spikes(spikes)

proc.run(500)
v = proc.neuron_vectors()
for i in range(len(v)):
    if len(v[i]) != 0:
        print(f'{i} fires at {v[i]}')

#proc.neuron_last_fires()

1 fires at [24.0, 27.0, 31.0, 34.0, 37.0, 40.0]
7 fires at [1.0, 2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0]
14 fires at [10.0]
19 fires at [5.0, 7.0, 10.0, 13.0, 15.0, 18.0, 21.0, 23.0]
32 fires at [4.0, 5.0, 6.0, 7.0, 9.0, 10.0, 11.0, 13.0, 14.0, 15.0, 16.0, 18.0, 19.0, 20.0, 22.0, 23.0]
45 fires at [9.0, 10.0, 12.0, 14.0, 16.0, 18.0, 20.0, 21.0, 23.0, 25.0, 27.0, 30.0]
48 fires at [8.0, 10.0, 13.0, 16.0, 19.0, 22.0, 25.0]
51 fires at [15.0, 18.0, 22.0, 26.0, 28.0, 30.0, 31.0, 33.0, 34.0, 36.0, 39.0, 41.0, 44.0]
61 fires at [15.0, 16.0, 18.0, 20.0, 22.0, 24.0, 26.0, 27.0, 29.0, 31.0, 33.0, 36.0]
73 fires at [9.0, 11.0, 13.0, 15.0, 17.0, 19.0, 21.0, 23.0, 26.0, 28.0]
75 fires at [3.0]
76 fires at [2.0, 4.0, 6.0, 8.0, 11.0, 13.0, 15.0, 18.0, 20.0, 26.0]
82 fires at [2.0, 4.0, 6.0, 8.0, 10.0, 12.0, 14.0, 17.0, 19.0, 21.0]
88 fires at [9.0, 10.0, 12.0, 14.0, 16.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 28.0, 30.0, 32.0, 34.0, 35.0, 37.0]
113 fires at [2.0]
116 

In [13]:
print(dir(proc))
proc.output_counts()

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_pybind11_conduit_v1_', 'apply_binary_data', 'apply_dvs_events', 'apply_spike', 'apply_spikes', 'clear', 'clear_activity', 'get_name', 'get_network_properties', 'get_params', 'get_processor_properties', 'get_time', 'load_network', 'load_networks', 'neuron_charges', 'neuron_counts', 'neuron_last_fires', 'neuron_vectors', 'output_count', 'output_count_all', 'output_count_max', 'output_counts', 'output_last_fire', 'output_last_fires', 'output_vector', 'output_vectors', 'run', 'synapse_weights', 'total_neuron_accumulates', 'total_neuron_counts', 'track_neuron_events', 'track_output_events']


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
# eons training
eons_inst.set_template_network(net)
pop = eons_inst.generate_population({'starting_edges': NUM_SYNAPSES, 'starting_nodes': NUM_NEURONS}, POP_SIZE)

#for n in pop.networks:
    #print(n.network.num_edges())

# for now let's say our fitness is # of correct predictions on training set
def compute_fitness(network):
    proc.load_network(network)

    correct = 0
    total = 0
    for sample in train_dataset:
        proc.clear_activity()
        spikes = encoder.get_spikes(sample[0])
        proc.apply_spikes(spikes)
        proc.run(2000)
        out_counts = proc.output_counts()

        prediction = out_counts.index(max(out_counts))

        if prediction == sample[1]:
            correct += 1
        total += 1

    return correct
    

for i in range(50):
    print(f'STARTING EPOCH {i}')
    fits = [compute_fitness(n.network) for n in pop.networks]
    print(f'Average accuracy of pop {(sum(fits)/len(fits))/DATASET_CAP:.2f}')
    pop = eons_inst.do_epoch(pop, fits, eons_param)

STARTING EPOCH 0
Average accuracy of pop 0.13
STARTING EPOCH 1
Average accuracy of pop 0.20
STARTING EPOCH 2
Average accuracy of pop 0.25
STARTING EPOCH 3
Average accuracy of pop 0.29
STARTING EPOCH 4
Average accuracy of pop 0.31
STARTING EPOCH 5
Average accuracy of pop 0.33
STARTING EPOCH 6
Average accuracy of pop 0.33
STARTING EPOCH 7
Average accuracy of pop 0.33
STARTING EPOCH 8
Average accuracy of pop 0.33
STARTING EPOCH 9


KeyboardInterrupt: 